In [ ]:
%load_ext autoreload
%autoreload 2
%run __init__
rs1_path = path['RSRCH'](1)

cache_path = './cache/e001'
try:
    os.mkdir(cache_path)
except:
    pass

fig_path = './figures/e001'
try:
    os.mkdir(fig_path)
except:
    pass

import importlib
e001_LL = importlib.import_module('e001-ANK-Line_Length')
e001_SPK = importlib.import_module('e001-ANK-NPSpikeDetect')

stdout_orig = sys.stdout

flatui = ["#46637f", "#2ecc71", "#e74c3c", "#3498db", "#9b59b6", "#95a5a6"]
sns.palplot(sns.color_palette("tab10"))
sns.set_palette("tab10")

In [ ]:
df_npref = path['CORE']['RNS']['NP_Ref']
df_ctlg = path['CORE']['RNS']['CATALOG']
TOD_DAYTIME = (7, 19)
BASELINE_DAY = 90

# Line-Length Overlay

In [ ]:
ix = 2000
ch_ix = 2

sel = df_ctlg[df_ctlg['ECoG trigger'] == 'Scheduled'].iloc[[ix]]
pth_LL = '{}.npz'.format(e001_LL.linelength_name(sel.iloc[0]['NP_code'], sel.iloc[0]['Filename']))

# Load raw data
df_raw = utils.neuropaceIO.get_ieeg_data_dict(sel, path['CORE']['RNS']['BASE'])
utils.signal_dict.zscore(df_raw, 'sample', method='robust')

# Load preproc Line Length
df_ll = utils.signal_dict.load_data_dict(pth_LL)
Fs = utils.signal_dict.get_fs(df_ll)
utils.signal_dict.zscore(df_ll, 'sample', method='robust')

# Reset time
t_raw = df_raw['sample']['timestamp']-df_raw['sample']['timestamp'][0]
t_ll = df_ll['sample']['timestamp']-df_ll['sample']['timestamp'][0]

# Generate the plot
%matplotlib inline
plt.figure(figsize=(6,3), dpi=300)
ax = plt.subplot(111)
ax.plot(t_raw, df_raw['signal'][:, ch_ix], linewidth=0.5)
ax.plot(t_ll, df_ll['signal'][:, 0, ch_ix], linewidth=0.5)

# PLot
ax.set_title('{}'.format(df_ll['channel']['label'][ch_ix]), fontsize=10)
ax.set_xlabel('Time in Clip (sec)')
ax.set_ylabel('Amplitude (z.s.)')

legend = ax.legend(['iEEG', 'Line-Length'])
legend.get_frame().set_linewidth(0)

plt.xlim([9, 11]);
plt.savefig('{}/Spikes.{}.{}.svg'.format(fig_path, sel.iloc[0]['Initials'], df_ll['channel']['label'][ch_ix]))
plt.show()

# Line-Length Validation

## Generate Multi-Subject Clips (blinded + randomized)

In [ ]:
import matplotlib.gridspec as gridspec

sel_npref = df_npref[df_npref['Responder_Type'].isin(['SR', 'NR'])].sample(frac=1, random_state=15).reset_index()
sel_npref[['NP_code', 'Responder_Type']].to_csv('{}/LL_Validation.NP_code.csv'.format(cache_path))

for np_rnd_id, sel_np in sel_npref.iterrows():
    np_code = sel_np['NP_code']    
    key_dates = extract_key_dates(np_code)
    XX, n_exclude = e001_LL.resample_LL(np_code, remove_blank=True) 
    YY, n_exclude = e001_SPK.resample_SPK(np_code, remove_blank=True) 
        
    sel_XX = XX.loc[XX.sort_values(by='mean').index[np.concatenate(
        (np.arange(10), np.arange(-10, 0)))]]
    sel_YY = YY.loc[sel_XX.index]
    
    sel_ctlg = df_ctlg[df_ctlg['NP_code'] == np_code]
    sel_ctlg['Raw UTC Timestamp'] = pd.to_datetime(sel_ctlg['Raw UTC Timestamp'], errors='coerce')
    sel_ctlg = sel_ctlg.set_index('Raw UTC Timestamp')

    sel_ctlg_rand = sel_ctlg.loc[sel_XX.index].sample(frac=1, random_state=15)
    sel_XX_rand = sel_XX.loc[sel_ctlg_rand.index]
    sel_YY_rand = sel_YY.loc[sel_ctlg_rand.index]
    
    sel_ctlg_rand = sel_ctlg_rand.reset_index()
    sel_XX_rand = sel_XX_rand.reset_index()
    sel_YY_rand = sel_YY_rand.reset_index()
    
    sel_XX_rand.to_pickle('{}/LL_Validation.LLmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))
    sel_YY_rand.to_pickle('{}/LL_Validation.SPKmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))    
    sel_ctlg_rand.to_pickle('{}/LL_Validation.Catalog_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))

    ### Generate the plot for annotation
    fig = plt.figure(figsize=(36,18), dpi=300)
    grid_outer = gridspec.GridSpec(5, 4, wspace=0.25, hspace=0.4)

    for sel_i, sel in sel_ctlg_rand.iterrows(): 

        # Load raw data
        df_raw = utils.neuropaceIO.get_ieeg_data_dict(sel_ctlg_rand.iloc[[sel_i]], path['CORE']['RNS']['BASE'])
        Fs = utils.signal_dict.get_fs(df_raw)
        df_raw = utils.signal_dict.subset(df_raw, sample=slice(0, int(e001_LL.PRE_TRIGGER_DUR*Fs)))
        utils.signal_dict.zscore(df_raw, 'sample', method='robust')

        # Reset time
        t_raw = np.arange(0, df_raw['signal'].shape[0]) / Fs

        # Generate the plot
        grid_inner = gridspec.GridSpecFromSubplotSpec(4, 1,
                        subplot_spec=grid_outer[sel_i], wspace=0.1, hspace=0.3)

        ax = None
        for ii in range(4):
            ax = plt.Subplot(fig, grid_inner[ii], sharey=ax)
            ax.plot(t_raw, df_raw['signal'][:, ii], linewidth=0.25)
            ax.set_xlim([0, 30])
            ax.set_ylim([-5, 5])

            ax.set_ylabel(df_raw['channel']['label'][ii], rotation=0, labelpad=30)
            if ii != 3:
                ax.set_xticklabels([])
            if ii == 3:
                ax.set_xlabel('Time (sec)')
            if ii == 0:
                ax.set_title('Clip ID: {}_{:0>2}'.format(np_rnd_id, sel_i))
            fig.add_subplot(ax)
    plt.tight_layout()
    plt.savefig('{}/LL_RATE.{}.pdf'.format(fig_path, np_rnd_id))
    plt.close()

### Combine Features + Gold-Standard Annots

In [ ]:
np_validation_LUT = pd.read_csv('{}/LL_Validation.NP_code.csv'.format(cache_path))

validation_dict = {'NP_code': [],
                   'Responder_Type': [],
                   'meanLL': [],
                   'NPSPK': [],
                   'IEA': [],
                   'SZ': []}
for np_rnd_id, np_valid in np_validation_LUT.iterrows():
    try:
        np_annot = pd.read_excel('{}/Subject.{}.VR_Annot.xlsx'.format(cache_path, np_rnd_id))
        np_LL = pd.read_pickle('{}/LL_Validation.LLmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_valid['NP_code'], np_valid['Responder_Type']))
        np_SPK = pd.read_pickle('{}/LL_Validation.SPKmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_valid['NP_code'], np_valid['Responder_Type']))
        #sel_ctlg_rand = pd.read_pickle('{}/LL_Validation.Catalog_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))
    except:
        continue
    
    for ii in range(np_annot.shape[0]):
        validation_dict['NP_code'].append(np_valid['NP_code'])
        validation_dict['Responder_Type'].append(np_valid['Responder_Type'])
        validation_dict['meanLL'].append(np_LL.iloc[ii]['CoVR'])
        validation_dict['NPSPK'].append(np_SPK.iloc[ii][7.5])                
        validation_dict['IEA'].append(np_annot.iloc[ii]['IEA'])
        validation_dict['SZ'].append(np_annot.iloc[ii]['SZ'])
validation_dict = pd.DataFrame.from_dict(validation_dict)

#### ROC (Line Length)

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(6,4), dpi=150)
ax = plt.subplot(111)

auc_null = []
for i in range(1000):
    fpr, tpr, _ = roc_curve(validation_dict['IEA'].sample(frac=1), validation_dict['meanLL'])
    auc_null.append(auc(fpr, tpr))
    
    ax.plot(fpr, tpr, color='gray', lw=0.1, alpha=0.1)
    
fpr, tpr, _ = roc_curve(validation_dict['IEA'], validation_dict['meanLL'])
auc_true = auc(fpr, tpr)
ax.plot(fpr, tpr, color='darkorange', lw=2)

ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
plt.legend(['AUC: {:0.2f} (p={:0.3f})'.format(auc_true, np.mean(np.array(auc_null) > auc_true))])
plt.savefig('{}/IEA_Validation.LL_Detector.MultiSubject.svg'.format(fig_path))
plt.show()

#### ROC (Desai et. al., 2019)

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(6,4), dpi=150)
ax = plt.subplot(111)

auc_null = []
for i in range(1000):
    fpr, tpr, _ = roc_curve(validation_dict['IEA'].sample(frac=1), validation_dict['NPSPK'])
    auc_null.append(auc(fpr, tpr))
    
    ax.plot(fpr, tpr, color='gray', lw=0.1, alpha=0.1)
    
fpr, tpr, _ = roc_curve(validation_dict['IEA'], validation_dict['NPSPK'])
auc_true = auc(fpr, tpr)
ax.plot(fpr, tpr, color='darkorange', lw=2)

ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
plt.legend(['AUC: {:0.2f} (p={:0.3f})'.format(auc_true, np.mean(np.array(auc_null) > auc_true))])
plt.savefig('{}/IEA_Validation.STDEV_Detector.MultiSubject.svg'.format(fig_path))
plt.show()

## Generate Single-Subject Clips (blinded + randomized)

In [ ]:
import matplotlib.gridspec as gridspec

sel_npref = df_npref[df_npref['Responder_Type'].isin(['SR', 'NR'])].sample(frac=1, random_state=15).reset_index()
sel_npref[['NP_code', 'Responder_Type']].to_csv('{}/LL_Validation.NP_code.csv'.format(cache_path))
sel_npref = sel_npref[sel_npref['NP_code'] == 'NP31']

for np_rnd_id, sel_np in sel_npref.iterrows():
    try:
        os.mkdir('{}/IEA_Annots/Randomized_SchedRec.{}'.format(fig_path, np_rnd_id))
    except:
        pass

    np_code = sel_np['NP_code']    
    key_dates = extract_key_dates(np_code)
    XX, n_exclude = e001_LL.resample_LL(np_code, remove_blank=True) 
    YY, n_exclude = e001_SPK.resample_SPK(np_code, remove_blank=True) 
        
    sel_ctlg = df_ctlg[df_ctlg['NP_code'] == np_code]
    sel_ctlg['Raw UTC Timestamp'] = pd.to_datetime(sel_ctlg['Raw UTC Timestamp'], errors='coerce')
    sel_ctlg = sel_ctlg.set_index('Raw UTC Timestamp')

    sel_ctlg_rand = sel_ctlg.loc[XX.index].sample(frac=1, random_state=15)
    sel_XX_rand = XX.loc[sel_ctlg_rand.index]
    sel_YY_rand = YY.loc[sel_ctlg_rand.index]
    
    sel_ctlg_rand = sel_ctlg_rand.reset_index()
    sel_XX_rand = sel_XX_rand.reset_index()
    sel_YY_rand = sel_YY_rand.reset_index()    
    
    sel_XX_rand.to_pickle('{}/LL_Validation.All_Sched.LLmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))
    sel_YY_rand.to_pickle('{}/LL_Validation.All_Sched.SPKmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))    
    sel_ctlg_rand.to_pickle('{}/LL_Validation.All_Sched.Catalog_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))

    for sel_i, sel in sel_ctlg_rand.iterrows(): 
        page_id = sel_i // 5  
        clip_row = (sel_i % 5)

        if clip_row == 0:
            ### Generate the plot for annotation
            fig = plt.figure(figsize=(24,18), dpi=300)
            grid_outer = gridspec.GridSpec(5, 1, wspace=0.25, hspace=0.4)
        
        # Load raw data
        df_raw = utils.neuropaceIO.get_ieeg_data_dict(sel_ctlg_rand.iloc[[sel_i]], path['CORE']['RNS']['BASE'])
        Fs = utils.signal_dict.get_fs(df_raw)
        df_raw = utils.signal_dict.subset(df_raw, sample=slice(0, int(e001_LL.PRE_TRIGGER_DUR*Fs)))
        utils.signal_dict.zscore(df_raw, 'sample', method='robust')

        # Reset time
        t_raw = np.arange(0, df_raw['signal'].shape[0]) / Fs

        # Generate the plot
        grid_inner = gridspec.GridSpecFromSubplotSpec(4, 1,
                        subplot_spec=grid_outer[clip_row], wspace=0.1, hspace=0.3)

        ax = None
        for ii in range(4):
            ax = plt.Subplot(fig, grid_inner[ii], sharey=ax)
            ax.plot(t_raw, df_raw['signal'][:, ii], linewidth=0.25)
            ax.set_xlim([0, 30])
            ax.set_ylim([-7, 7])

            if ii != 3:
                ax.set_xticklabels([])
            if ii == 3:
                ax.set_xlabel('Time (sec)')
            if ii == 0:
                ax.set_title('Clip ID: {}_{:0>2}'.format(np_rnd_id, sel_i))
            fig.add_subplot(ax)
        
        if clip_row == 4:
            plt.tight_layout()
            plt.savefig('{}/IEA_Annots/Randomized_SchedRec.{}/Randomized_SchedRec.{}.Page_{:0>3}.pdf'.format(fig_path, np_rnd_id, np_rnd_id, page_id))
            plt.close()
    plt.tight_layout()
    plt.savefig('{}/IEA_Annots/Randomized_SchedRec.{}/Randomized_SchedRec.{}.Page_{:0>3}.pdf'.format(fig_path, np_rnd_id, np_rnd_id, page_id))
    plt.close()

### Combine Features + Gold-Standard Annots

In [ ]:
np_validation_LUT = pd.read_csv('{}/LL_Validation.NP_code.csv'.format(cache_path))

validation_dict = {'NP_code': [],
                   'Responder_Type': [],
                   'meanLL': [],
                   'NPSPK': [],
                   'IEA': [],
                   'SZ': []}
for np_rnd_id, np_valid in np_validation_LUT.iterrows():
    try:
        np_annot = pd.read_excel('{}/IEA_Annots/Randomized_SchedRec.{}/Subject.{}.VR_Annot_scored.xlsx'.format(fig_path, np_rnd_id, np_rnd_id))
        np_LL = pd.read_pickle('{}/LL_Validation.All_Sched.LLmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_valid['NP_code'], np_valid['Responder_Type']))
        np_SPK = pd.read_pickle('{}/LL_Validation.All_Sched.SPKmean_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_valid['NP_code'], np_valid['Responder_Type']))        
        #sel_ctlg_rand = pd.read_pickle('{}/LL_Validation.All_Sched.Catalog_perm.{}.{}.{}.pkl'.format(cache_path, np_rnd_id, np_code, sel_np['Responder_Type']))
    except:
        continue
    
    for ii in range(np_annot.shape[0]):
        validation_dict['NP_code'].append(np_valid['NP_code'])
        validation_dict['Responder_Type'].append(np_valid['Responder_Type'])
        validation_dict['meanLL'].append(np_LL.iloc[ii]['CoVR'])
        validation_dict['NPSPK'].append(np_SPK.iloc[ii][7.5])        
        validation_dict['IEA'].append(np_annot.iloc[ii]['IEA'])
        validation_dict['SZ'].append(np_annot.iloc[ii]['SZ'])
validation_dict = pd.DataFrame.from_dict(validation_dict)

#### ROC (Line Length)

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(6,4), dpi=150)
ax = plt.subplot(111)

auc_null = []
for i in range(1000):
    fpr, tpr, _ = roc_curve(validation_dict.groupby(['NP_code'])['IEA'].transform(np.random.permutation), validation_dict['meanLL'])
    auc_null.append(auc(fpr, tpr))
    
    ax.plot(fpr, tpr, color='gray', lw=0.1, alpha=0.25)
    
fpr, tpr, _ = roc_curve(validation_dict['IEA'], validation_dict['meanLL'])
auc_true = auc(fpr, tpr)
ax.plot(fpr, tpr, color='darkorange', lw=2)

ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
plt.legend(['AUC: {:0.2f} (p={:0.3f})'.format(auc_true, np.mean(np.array(auc_null) > auc_true))])
plt.savefig('{}/IEA_Validation.LL_Detector.SingleSubject.{}.svg'.format(fig_path, validation_dict['NP_code'].unique()[0]))
plt.show()

#### ROC (Desai et. al., 2019)

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(6,4), dpi=150)
ax = plt.subplot(111)

auc_null = []
for i in range(1000):
    fpr, tpr, _ = roc_curve(validation_dict.groupby(['NP_code'])['IEA'].transform(np.random.permutation), validation_dict['NPSPK'])
    auc_null.append(auc(fpr, tpr))
    
    ax.plot(fpr, tpr, color='gray', lw=0.1, alpha=0.25)
    
fpr, tpr, _ = roc_curve(validation_dict['IEA'], validation_dict['NPSPK'])
auc_true = auc(fpr, tpr)
ax.plot(fpr, tpr, color='darkorange', lw=2)

ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
plt.legend(['AUC: {:0.2f} (p={:0.3f})'.format(auc_true, np.mean(np.array(auc_null) > auc_true))])
plt.savefig('{}/IEA_Validation.STDEV_Detector.SingleSubject.{}.svg'.format(fig_path, validation_dict['NP_code'].unique()[0]))
plt.show()

# Time of Day Sampling Statistics

In [ ]:
CLIP_STATS = []
for np_code in df_npref['NP_code'].unique():
    print(np_code)
    
    try:
        key_dates = extract_key_dates(np_code)

        #####      
        X, n_exclude = e001_LL.resample_LL(np_code, remove_blank=False)        
    except Exception as E:
        print(1)
        print(E)
        continue

    # Resample temporal data
    duration = np.nan
    try:              
        X = X.loc[X.index >= key_dates[0]].dropna()   
        X_daytime = X.loc[(((X.index.hour) >= TOD_DAYTIME[0]) & 
                           ((X.index.hour) < TOD_DAYTIME[1]))].dropna()
        X_nighttime = X.loc[~(((X.index.hour) >= TOD_DAYTIME[0]) & 
                             ((X.index.hour) < TOD_DAYTIME[1]))].dropna()
        
        for epoch, n_rec in [('All', len(X)),
                             ('Daytime', len(X_daytime)),
                             ('Nighttime', len(X_nighttime))]:
            CLIP_STATS.append(
                pd.DataFrame.from_dict({'Epoch': epoch,
                                        'N_Recording': n_rec,
                                        'NP_code': [np_code]}).set_index('NP_code')
            )
        
        
    except Exception as E:
        print(2)
        print(E)
        continue

        
        
####
CLIP_STATS = pd.concat(CLIP_STATS)
CLIP_STATS = pd.merge(CLIP_STATS, df_npref[['NP_code', 'Responder_Type']].set_index('NP_code'),
                      left_index=True, right_index=True)

In [ ]:
plt.figure(figsize=(4,6), dpi=300)
ax = plt.subplot(111)
ax = sns.boxplot(x='Responder_Type', y='N_Recording', fliersize=0, order=['SR', 'IR', 'NR'], data=CLIP_STATS[CLIP_STATS['Epoch'] == 'All'], ax=ax);
ax = sns.swarmplot(x='Responder_Type', y='N_Recording', color='k', order=['SR', 'IR', 'NR'], data=CLIP_STATS[CLIP_STATS['Epoch'] == 'All'], ax=ax);
Hv, pv = sp_stats.kruskal(CLIP_STATS[(CLIP_STATS['Epoch'] == 'All') & (CLIP_STATS['Responder_Type'] == 'SR')]['N_Recording'],
                          CLIP_STATS[(CLIP_STATS['Epoch'] == 'All') & (CLIP_STATS['Responder_Type'] == 'IR')]['N_Recording'],
                          CLIP_STATS[(CLIP_STATS['Epoch'] == 'All') & (CLIP_STATS['Responder_Type'] == 'NR')]['N_Recording'])
ax.text(1.5, 3500, s='H={:0.2f}\np={:0.3f}'.format(Hv, pv))
ax.set_title('All Scheduled Clips')
plt.savefig('{}/Responder_ScheduledSampling.N_Total_Clips.svg'.format(fig_path))
plt.show()


plt.figure(figsize=(4,6), dpi=300)
ax = plt.subplot(111)
ax = sns.boxplot(x='Responder_Type', y='N_Recording', fliersize=0, hue='Epoch', order=['SR', 'IR', 'NR'], data=CLIP_STATS[~(CLIP_STATS['Epoch'] == 'All')], ax=ax);
print(CLIP_STATS.groupby('Responder_Type').apply(
    lambda x: sp_stats.wilcoxon(x[x['Epoch'] == 'Daytime']['N_Recording'],
                                x[x['Epoch'] == 'Nighttime']['N_Recording'])))
plt.savefig('{}/Responder_ScheduledSampling.N_DayNight_Clips.svg'.format(fig_path))
plt.show()

# Identify Long-Term Line-Length Changes

In [ ]:
# Grab data for NP_code    
X_LL_Subject = {}

# Baseline Period
for np_code in df_npref['NP_code'].unique():
    print(np_code)
    
    try:
        key_dates = extract_key_dates(np_code)

        #####      
        XX, n_exclude = e001_LL.resample_LL(np_code, remove_blank=True)        
    except Exception as E:
        print(1)
        print(E)
        continue
    
    for metric in XX.columns:
        if metric not in X_LL_Subject:
            X_LL_Subject[metric] = {}
                
        for TOD in ['24h', 'Daytime', 'Nighttime']:
            if TOD not in X_LL_Subject[metric]:
                X_LL_Subject[metric][TOD] = []

            # Resample temporal data
            try:         
                X = XX.copy()
                
                X = X.loc[X.index >= key_dates[0]]   
                if TOD == 'Daytime':
                    X = X.loc[(((X.index.hour) >= TOD_DAYTIME[0]) & 
                               ((X.index.hour) < TOD_DAYTIME[1]))]
                elif TOD == 'Nighttime':
                    X = X.loc[~(((X.index.hour) >= TOD_DAYTIME[0]) & 
                                ((X.index.hour) < TOD_DAYTIME[1]))]

                X = X.set_index(X.index - key_dates[0])  
                X0 = np.nan*X.iloc[[0]]
                X0.index = [pd.Timedelta('0D')]
                X = X0.append(X).resample('30D').mean().rolling('{}D'.format(BASELINE_DAY)).mean()
                X = X.interpolate()

                X_LL_Subject[metric][TOD].append(X[metric].to_frame(name=np_code))
            except Exception as E:
                print(2)
                print(E)
            continue

for metric in X_LL_Subject:
    for TOD in X_LL_Subject[metric]:
        X_LL_Subject[metric][TOD] = pd.concat(X_LL_Subject[metric][TOD], axis=1)

## Baseline Differences

In [ ]:
for metric in ['CoVR', 'Z=3', 'Z=5']:
    for TOD in X_LL_Subject[metric]:
        LL_BASELINE = X_LL_Subject[metric][TOD].iloc[0].to_frame(name='Early')

        combined = df_npref[['N_Vs_MT', 'Responder_Type', 'NP_code']].set_index('NP_code').join(LL_BASELINE)
        combined = combined[combined['N_Vs_MT'].isin(['N', 'M', 'B'])]

        plt.figure(figsize=(3,6), dpi=100)
        ax = plt.subplot(111)
        ax = sns.boxplot(x='Responder_Type', y='Early', order=['NR', 'SR'], fliersize=0, data=combined, ax=ax);
        ax = sns.swarmplot(x='Responder_Type', y='Early', order=['NR', 'SR'], color='k', data=combined, ax=ax);
        tv, pv = sp_stats.ranksums(combined[combined['Responder_Type'] == 'NR']['Early'].dropna(),
                                    combined[combined['Responder_Type'] == 'SR']['Early'].dropna())
        print('Early: {:0.2f}, {:0.3f}'.format(tv, pv))
        #ax.text(1, 90, s='t={:0.2f}\np={:0.2f}'.format(tv, pv))
        ax.set_ylabel('Line-Length ({})'.format(metric))
        ax.set_ylim(ymin=0.0)
        ax.set_title('{}'.format(TOD))

        plt.tight_layout()
        plt.savefig('{}/Baseline.LL.{}-{}.svg'.format(fig_path, metric, TOD))
        plt.show()

## Trajectory

In [ ]:
for metric in ['CoVR', 'Z=3', 'Z=5']:
    for TOD in X_LL_Subject[metric]:
        LL_PCT_CHANGE = 100*(X_LL_Subject[metric][TOD] / X_LL_Subject[metric][TOD].iloc[0] - 1)
        LL_PCT_CHANGE = LL_PCT_CHANGE[LL_PCT_CHANGE.index <= '720 days']
        LL_PCT_CHANGE = LL_PCT_CHANGE.replace([np.inf, -np.inf], np.nan)

        days = np.array(LL_PCT_CHANGE.index.days)
        sel_npref = df_npref[['N_Vs_MT', 'Responder_Type', 'NP_code']]
        #sel_npref = sel_npref[~sel_npref['NP_code'].isin(path['CORE']['RNS']['NP_LOC'])]
        sel_npref = sel_npref[sel_npref['NP_code'].isin(LL_PCT_CHANGE.columns)]  

        ####
        sel_npref = sel_npref[sel_npref['N_Vs_MT'].isin(['N', 'M', 'B'])]
        sel_npref = sel_npref[sel_npref['Responder_Type'].isin(['SR', 'NR'])]

        SR_Subj = sel_npref[sel_npref['Responder_Type'] == 'SR']['NP_code']
        NR_Subj = sel_npref[sel_npref['Responder_Type'] == 'NR']['NP_code']

        ####
        SR_mean = np.nanmean(LL_PCT_CHANGE[SR_Subj], axis=1)
        SR_sem = np.nanstd(LL_PCT_CHANGE[SR_Subj], axis=1) / np.sqrt(len(SR_Subj))

        NR_mean = np.nanmean(LL_PCT_CHANGE[NR_Subj], axis=1)
        NR_sem = np.nanstd(LL_PCT_CHANGE[NR_Subj], axis=1) / np.sqrt(len(NR_Subj))
        
        #### FUNCTIONAL DATA ANALYSIS
        true_diff = np.nanmean(SR_mean[1:] - NR_mean[1:])
        null_diff = []
        for rnd in range(10000):
            sel_npref_rnd = sel_npref.copy()
            sel_npref_rnd['Responder_Type'] = np.random.permutation(sel_npref['Responder_Type'])
            SR_Subj_rnd = sel_npref_rnd[sel_npref_rnd['Responder_Type'] == 'SR']['NP_code']
            NR_Subj_rnd = sel_npref_rnd[sel_npref_rnd['Responder_Type'] == 'NR']['NP_code']
            SR_mean_rnd = np.nanmean(LL_PCT_CHANGE[SR_Subj_rnd], axis=1)
            NR_mean_rnd = np.nanmean(LL_PCT_CHANGE[NR_Subj_rnd], axis=1)
            null_diff.append(np.nanmean(SR_mean_rnd[1:] - NR_mean_rnd[1:]))
        if true_diff > 0:
            fda_pv = np.mean(np.array(null_diff) > true_diff)
        else:
            fda_pv = np.mean(np.array(null_diff) < true_diff)
        print('Functional Data Analysis: {}'.format(fda_pv))

        #### T-Stat
        sig_days = []
        for ix in LL_PCT_CHANGE.index:
            try:
                tv, pv = sp_stats.ranksums(LL_PCT_CHANGE[SR_Subj].loc[ix].dropna(),
                                           LL_PCT_CHANGE[NR_Subj].loc[ix].dropna(),
                                           )

            except Exception as E:
                print(E)
                tv, pv = [0, 1]

            sig_days.append([ix.days, pv])
        sig_days = np.array(sig_days)
        sig_days = sig_days[sig_days[:,1] < 0.05][:, 0]

        #### Plot
        plt.figure(figsize=(12,4), dpi=300)
        ax = plt.subplot(1,1,1)  

        ax.plot(days, SR_mean, color='g')
        ax.fill_between(days, SR_mean-SR_sem, SR_mean+SR_sem, color='g', alpha=0.1)
        ax.plot(days, NR_mean, color='r')
        ax.fill_between(days, NR_mean-NR_sem, NR_mean+NR_sem, color='r', alpha=0.1)
        ax.scatter(sig_days, 1.1*np.max(SR_mean)*np.ones(len(sig_days)), marker='.', color='k')

        ax.set_xticks(days)
        ax.hlines(0, 0, LL_PCT_CHANGE.index[-1].days, linestyle='--', color='k')
        ax.set_xlim([0, LL_PCT_CHANGE.index[-1].days])
        ax.set_ylim([-25, 25])

        ax.set_xlabel('Time Post-Implant (Days)')
        ax.set_ylabel('Percent Change in Line-Length ({})'.format(metric))  
        ax.set_title('{}'.format(TOD))

        plt.tight_layout()
        plt.savefig('{}/Trajectory.LL.{}-{}.svg'.format(fig_path, metric, TOD))
        plt.show()